# **Problem 1**

**Algorithm 1**: Priority-Based Scheduling (Combined with Elements of Greedy Algorithm)

In [ ]:
# Define the availability of each staff member for each time slot
staff_availability = {
    'Dr. Smith': ['Monday Morning', 'Monday Afternoon', 'Monday Night',
                  'Wednesday Morning', 'Wednesday Afternoon', 'Wednesday Night',
                  'Friday Morning', 'Friday Afternoon', 'Friday Night'],
    'Dr. Johnson': ['Tuesday Morning', 'Tuesday Afternoon', 'Tuesday Night',
                    'Thursday Morning', 'Thursday Afternoon', 'Thursday Night'],
    'Nurse Brown': ['Monday Morning', 'Monday Afternoon', 'Monday Night',
                    'Tuesday Morning', 'Tuesday Afternoon', 'Tuesday Night',
                    'Wednesday Morning', 'Wednesday Afternoon', 'Wednesday Night',
                    'Thursday Morning', 'Thursday Afternoon', 'Thursday Night',
                    'Friday Morning', 'Friday Afternoon', 'Friday Night'],
    'Nurse Miller': ['Monday Morning', 'Monday Afternoon', 'Monday Night',
                     'Wednesday Morning', 'Wednesday Afternoon', 'Wednesday Night',
                     'Friday Morning', 'Friday Afternoon', 'Friday Night']
}

# Define the availability of each room for each time slot
room_availability = {
    'Surgical Room': ['Monday Morning', 'Tuesday Morning', 'Wednesday Morning',
                      'Thursday Morning', 'Friday Morning'],
    'ICU': ['Tuesday Night', 'Thursday Night'],
    'Pediatrics': ['Monday Afternoon', 'Tuesday Afternoon', 'Wednesday Afternoon',
                   'Thursday Afternoon', 'Friday Afternoon']
}

# Define the patients and their needs
patients = {
    'P001': ('John Doe', 'Surgery'),
    'P002': ('Helen Jones', 'Pediatrics'),
    'P003': ('Emily Johnson', 'Cardiology'),
    'P004': ('Tom Miller', 'Surgery'),
    'P005': ('June Larson', 'Surgery')
}

# Define the mapping from diseases to rooms
disease_to_room = {
    'Surgery': 'Surgical Room',
    'Cardiology': 'ICU',
    'Pediatrics': 'Pediatrics'
}

# Define preferred staff for certain rooms
preferred_staff = {
    'Surgical Room': ('Dr. Johnson', 'Nurse Brown'),
    'ICU': ('Dr. Smith', ''),  # Preferred doctor for ICU is Dr. Smith
    'Pediatrics': ('', 'Nurse Miller')  # Preferred nurse for Pediatrics is Nurse Miller
}

# Initialize the schedule list
schedule = []

# Function to check if a staff member is available at a given time
def is_staff_available(name, time_slot):
    return time_slot in staff_availability.get(name, [])

# Function to get available staff
def get_available_staff(doctor_pref, nurse_pref, time_slot):
    # Try to get preferred staff first
    doctor = doctor_pref if is_staff_available(doctor_pref, time_slot) else None
    nurse = nurse_pref if is_staff_available(nurse_pref, time_slot) else None

    # If preferred doctor/nurse not available, get any available one
    if not doctor:
        doctor = next((d for d in staff_availability if 'Dr.' in d and is_staff_available(d, time_slot)), None)
    if not nurse:
        nurse = next((n for n in staff_availability if 'Nurse' in n and is_staff_available(n, time_slot)), None)

    return doctor, nurse

# Function to schedule a patient
def schedule_patient(patient_id, patient_name, disease):
    room = disease_to_room[disease]
    preferred_doctor, preferred_nurse = preferred_staff.get(room, (None, None))

    # Find the available time slot for the room
    for time_slot in room_availability[room]:
        # Get available doctor and nurse for the time slot
        doctor, nurse = get_available_staff(preferred_doctor, preferred_nurse, time_slot)

        # If both doctor and nurse are available, schedule the patient
        if doctor and nurse:
            schedule.append([patient_id, patient_name, room, time_slot, f"{doctor} and {nurse}"])

            # Mark the doctor and nurse as not available for this time slot
            staff_availability[doctor] = [t for t in staff_availability[doctor] if t != time_slot]
            staff_availability[nurse] = [t for t in staff_availability[nurse] if t != time_slot]

            # Break out of the loop once the patient is scheduled
            break

# Schedule all patients
for patient_id, (patient_name, disease) in patients.items():
    schedule_patient(patient_id, patient_name, disease)

# Output the schedule with aligned columns
print("{:<12}\t{:<15}\t{:<15}\t{:<20}\t{}".format("Patient ID", "Patient Name", "Room", "Time Slot", "Staff"))
for entry in schedule:
    print("{:<12}\t{:<15}\t{:<15}\t{:<20}\t{}".format(*entry))

Patient ID  	Patient Name   	Room           	Time Slot           	Staff
P001        	John Doe       	Surgical Room  	Monday Morning      	Dr. Smith and Nurse Brown
P002        	Helen Jones    	Pediatrics     	Monday Afternoon    	Dr. Smith and Nurse Miller
P003        	Emily Johnson  	ICU            	Tuesday Night       	Dr. Johnson and Nurse Brown
P004        	Tom Miller     	Surgical Room  	Tuesday Morning     	Dr. Johnson and Nurse Brown
P005        	June Larson    	Surgical Room  	Wednesday Morning   	Dr. Smith and Nurse Brown


**Algorithm 2**: Backtracking Algorithm with Priority Consideration

In [ ]:
from prettytable import PrettyTable

def is_valid_assignment(assignment, doctor, nurse, time):
    # Ensure medical staff are not double-booked
    for _, (d, n, t, _) in assignment.items():
        if (d == doctor or n == nurse) and t == time:
            return False
    return True

def find_schedule(patients, patient_names, staff_availability, time_slots, disease_to_room, current_assignment={}, current_index=0):
    if current_index == len(patients):
        return current_assignment

    patient, disease = patients[current_index]
    room = disease_to_room[disease]
    for time in time_slots[room]:
        day, _ = time.split()
        doctors = [d for d in staff_availability if 'Dr.' in d and day in staff_availability[d]]
        nurses = [n for n in staff_availability if 'Nurse' in n and day in staff_availability[n]]

        # For a specific room, prioritize certain medical staff
        if room == 'Surgical Room':
            preferred_doctors = ['Dr. Johnson'] if 'Dr. Johnson' in doctors else doctors
            preferred_nurses = ['Nurse Brown'] if 'Nurse Brown' in nurses else nurses
        elif room == 'Pediatrics':
            preferred_doctors = doctors
            preferred_nurses = ['Nurse Miller'] if 'Nurse Miller' in nurses else nurses
        else:
            preferred_doctors = doctors
            preferred_nurses = nurses

        for doctor in preferred_doctors:
            for nurse in preferred_nurses:
                if is_valid_assignment(current_assignment, doctor, nurse, time):
                    current_assignment[patient] = (doctor, nurse, time, room)
                    result = find_schedule(patients, patient_names, staff_availability, time_slots, disease_to_room, current_assignment, current_index + 1)
                    if result is not None:
                        return result
                    del current_assignment[patient]

    return None

# Define patient names
patient_names = {
    'P001': 'John Doe',
    'P002': 'Helen Jones',
    'P003': 'Emily Johnson',
    'P004': 'Tom Miller',
    'P005': 'June Larson'
}

# Define the working hours for medical staff and the operating hours for rooms
staff_availability = {
    'Dr. Smith': ['Monday', 'Wednesday', 'Friday'],
    'Dr. Johnson': ['Tuesday', 'Thursday'],
    'Nurse Brown': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
    'Nurse Miller': ['Monday', 'Wednesday', 'Friday']
}

time_slots = {
    'Surgical Room': ['Monday Morning', 'Tuesday Morning', 'Wednesday Morning',
                      'Thursday Morning', 'Friday Morning'],
    'ICU': ['Tuesday Night', 'Thursday Night'],
    'Pediatrics': ['Monday Afternoon', 'Tuesday Afternoon', 'Wednesday Afternoon',
                   'Thursday Afternoon', 'Friday Afternoon']
}

disease_to_room = {
    'Surgery': 'Surgical Room',
    'Cardiology': 'ICU',
    'Pediatrics': 'Pediatrics'
}

patients = [
    ('P001', 'Surgery'),
    ('P002', 'Pediatrics'),
    ('P003', 'Cardiology'),
    ('P004', 'Surgery'),
    ('P005', 'Surgery')
]

patient_names = {
    'P001': 'John Doe',
    'P002': 'Helen Jones',
    'P003': 'Emily Johnson',
    'P004': 'Tom Miller',
    'P005': 'June Larson'
}

# Generate the schedule
schedule = find_schedule(patients, patient_names, staff_availability, time_slots, disease_to_room)


# Print the scheduling results in the specified format
if schedule:
    print("{:<12}\t{:<15}\t{:<15}\t{:<20}\t{}".format("Patient ID", "Patient Name", "Room", "Time Slot", "Staff"))
    for patient, (doctor, nurse, time, room) in schedule.items():
        print("{:<12}\t{:<15}\t{:<15}\t{:<20}\t{} and {}".format(patient, patient_names[patient], room, time, doctor, nurse))
else:
    print("No schedule found.")

Patient ID  	Patient Name   	Room           	Time Slot           	Staff
P001        	John Doe       	Surgical Room  	Monday Morning      	Dr. Smith and Nurse Brown
P002        	Helen Jones    	Pediatrics     	Monday Afternoon    	Dr. Smith and Nurse Miller
P003        	Emily Johnson  	ICU            	Tuesday Night       	Dr. Johnson and Nurse Brown
P004        	Tom Miller     	Surgical Room  	Tuesday Morning     	Dr. Johnson and Nurse Brown
P005        	June Larson    	Surgical Room  	Wednesday Morning   	Dr. Smith and Nurse Brown


# **Problem 2**

**Algorithm 1**: Dijkstra’s Algorithm

In [ ]:
import heapq

# Define the weights based on quality and load
quality_to_weight = {
    'Poor': 4,
    'Fair': 3,
    'Good': 2,
    'Excellent': 1
}

load_to_weight = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

# Define the graph of the network as a dictionary
# The key is the node ID, and the value is a list of tuples (connected_node_id, combined_weight)
graph = {
    '001': [('002', quality_to_weight['Good'] + load_to_weight['High']),
            ('003', quality_to_weight['Excellent'] + load_to_weight['High']),
            ('004', quality_to_weight['Good'] + load_to_weight['High'])],
    '002': [('001', quality_to_weight['Good'] + load_to_weight['Medium']),
            ('003', quality_to_weight['Excellent'] + load_to_weight['Medium'])],
    '003': [('001', quality_to_weight['Excellent'] + load_to_weight['High']),
            ('002', quality_to_weight['Excellent'] + load_to_weight['High']),
            ('004', quality_to_weight['Poor'] + load_to_weight['High'])],
    '004': [('001', quality_to_weight['Good'] + load_to_weight['Low']),
            ('003', quality_to_weight['Poor'] + load_to_weight['Low'])]
}

def dijkstra(graph, start):
    min_heap = [(0, start)]  # Priority queue with (distance, node)
    distances = {node: float('infinity') for node in graph}  # Start with infinite distances
    distances[start] = 0  # The start node has distance 0 to itself
    paths = {node: [] for node in graph}  # To store the paths
    paths[start] = [start]

    while min_heap:
        current_distance, current_node = heapq.heappop(min_heap)

        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node]:
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(min_heap, (distance, neighbor))
                paths[neighbor] = paths[current_node] + [neighbor]

    return distances, paths

# Example usage:
start_node = '001'
end_node = '004'
distances, paths = dijkstra(graph, start_node)
print(f"Shortest path from {start_node} to {end_node} is: {paths[end_node]} with distance {distances[end_node]}\n")

def dijkstra_algorithm(graph, start_node):
    # Priority queue to maintain nodes to visit
    queue = [(0, start_node, [start_node])]  # (cumulative_weight, node, path)
    visited = set()
    paths = {}  # Store paths from start_node to each node

    while queue:
        (cumulative_weight, current_node, path) = heapq.heappop(queue)
        if current_node not in visited:
            visited.add(current_node)
            paths[current_node] = (cumulative_weight, path)
            for neighbor, weight in graph[current_node]:
                if neighbor not in visited:
                    heapq.heappush(queue, (cumulative_weight + weight, neighbor, path + [neighbor]))

    return paths

# Function to run Dijkstra's algorithm from each node
def dijkstra_all_routes(graph):
    all_routes = {}
    for node in graph:
        distances, paths = dijkstra(graph, node)
        all_routes[node] = {"distances": distances, "paths": paths}
    return all_routes

# Executing Dijkstra's algorithm for all nodes and storing all routes
dijkstra_routes = dijkstra_all_routes(graph)

# Displaying all routes with total weight from each node to every other node using Dijkstra's algorithm
print("Dijkstra's Algorithm Possible Routes with Total Weight")
for start_node, data in dijkstra_routes.items():
    print(f"From Node {start_node}:")
    for end_node, distance in data["distances"].items():
        path = data["paths"][end_node]
        print(f"  To {end_node}: Path: {path}, Total weight: {distance}")
    print()

Shortest path from 001 to 004 is: ['001', '004'] with distance 5

Dijkstra's Algorithm Possible Routes with Total Weight
From Node 001:
  To 001: Path: ['001'], Total weight: 0
  To 002: Path: ['001', '002'], Total weight: 5
  To 003: Path: ['001', '003'], Total weight: 4
  To 004: Path: ['001', '004'], Total weight: 5

From Node 002:
  To 001: Path: ['002', '001'], Total weight: 4
  To 002: Path: ['002'], Total weight: 0
  To 003: Path: ['002', '003'], Total weight: 3
  To 004: Path: ['002', '001', '004'], Total weight: 9

From Node 003:
  To 001: Path: ['003', '001'], Total weight: 4
  To 002: Path: ['003', '002'], Total weight: 4
  To 003: Path: ['003'], Total weight: 0
  To 004: Path: ['003', '004'], Total weight: 7

From Node 004:
  To 001: Path: ['004', '001'], Total weight: 3
  To 002: Path: ['004', '001', '002'], Total weight: 8
  To 003: Path: ['004', '003'], Total weight: 5
  To 004: Path: ['004'], Total weight: 0



**Algorithm 2**: Bellman-Ford Algorithm

In [ ]:
import heapq

# Define the weights based on quality and load
quality_to_weight = {
    'Poor': 4,
    'Fair': 3,
    'Good': 2,
    'Excellent': 1
}

load_to_weight = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

# Define the graph of the network as a dictionary
# The key is the node ID, and the value is a list of tuples (connected_node_id, combined_weight)
graph = {
    '001': [('002', quality_to_weight['Good'] + load_to_weight['High']),
            ('003', quality_to_weight['Excellent'] + load_to_weight['High']),
            ('004', quality_to_weight['Good'] + load_to_weight['High'])],
    '002': [('001', quality_to_weight['Good'] + load_to_weight['Medium']),
            ('003', quality_to_weight['Excellent'] + load_to_weight['Medium'])],
    '003': [('001', quality_to_weight['Excellent'] + load_to_weight['High']),
            ('002', quality_to_weight['Excellent'] + load_to_weight['High']),
            ('004', quality_to_weight['Poor'] + load_to_weight['High'])],
    '004': [('001', quality_to_weight['Good'] + load_to_weight['Low']),
            ('003', quality_to_weight['Poor'] + load_to_weight['Low'])]
}

def bellman_ford(graph, start):
    distances = {node: float('infinity') for node in graph}  # Start with infinite distances
    distances[start] = 0  # The start node has distance 0 to itself
    paths = {node: [] for node in graph}  # To store the paths
    paths[start] = [start]

    # Relax edges repeatedly
    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor, weight in graph[node]:
                if distances[node] + weight < distances[neighbor]:
                    distances[neighbor] = distances[node] + weight
                    paths[neighbor] = paths[node] + [neighbor]

    return distances, paths

# Example usage:
start_node = '001'
end_node = '004'
distances, paths = bellman_ford(graph, start_node)
print(f"Shortest path from {start_node} to {end_node} is: {paths[end_node]} with distance {distances[end_node]}\n")

# Function to run Bellman-Ford algorithm from each node
def bellman_ford_all_routes(graph):
    all_routes = {}
    for node in graph:
        distances, paths = bellman_ford(graph, node)
        all_routes[node] = {"distances": distances, "paths": paths}
    return all_routes

# Executing Bellman-Ford algorithm for all nodes and storing all routes
bellman_ford_routes = bellman_ford_all_routes(graph)

# Displaying all routes with total weight from each node to every other node using Bellman-Ford algorithm
print("Bellman-Ford Algorithm Possible Routes with Total Weight")
for start_node, data in bellman_ford_routes.items():
    print(f"From Node {start_node}:")
    for end_node, distance in data["distances"].items():
        path = data["paths"][end_node]
        print(f"  To {end_node}: Path: {path}, Total weight: {distance}")
    print()

Shortest path from 001 to 004 is: ['001', '004'] with distance 5

Bellman-Ford Algorithm Possible Routes with Total Weight
From Node 001:
  To 001: Path: ['001'], Total weight: 0
  To 002: Path: ['001', '002'], Total weight: 5
  To 003: Path: ['001', '003'], Total weight: 4
  To 004: Path: ['001', '004'], Total weight: 5

From Node 002:
  To 001: Path: ['002', '001'], Total weight: 4
  To 002: Path: ['002'], Total weight: 0
  To 003: Path: ['002', '003'], Total weight: 3
  To 004: Path: ['002', '001', '004'], Total weight: 9

From Node 003:
  To 001: Path: ['003', '001'], Total weight: 4
  To 002: Path: ['003', '002'], Total weight: 4
  To 003: Path: ['003'], Total weight: 0
  To 004: Path: ['003', '004'], Total weight: 7

From Node 004:
  To 001: Path: ['004', '001'], Total weight: 3
  To 002: Path: ['004', '001', '002'], Total weight: 8
  To 003: Path: ['004', '003'], Total weight: 5
  To 004: Path: ['004'], Total weight: 0

